In [1]:
import pandas as panda

from sklearn import metrics
from sklearn.model_selection import train_test_split




from google.colab import drive
drive.mount('/content/drive')

df = panda.read_csv('/content/drive/MyDrive/feature_complete6.0manhattenyellowtaxis.csv')



print(df.head())







Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   DOLocationID  dropoff_hour  DOLocation_area  DOLocation_perimeter  \
0           140            13         0.000114              0.047584   
1           243            13         0.000438              0.094331   
2            41            14         0.000143              0.052793   
3            48            14         0.000094              0.043747   
4            68            14         0.000111              0.049337   

   dropoff_day_of_week  dropoff_day_of_month  dropoff_month  busyness  
0                    0                    31             12         1  
1                    0                    31             12         1  
2                    0                    31             12         1  
3                    0                    31             12         1  
4                    0                    31             12         1  


In [2]:
#make the features into more efficient datatypes

df['DOLocationID'] =  df['DOLocationID'].astype('uint16')
df['dropoff_hour'] =   df['dropoff_hour'].astype('uint8')

df['dropoff_day_of_week'] =   df['dropoff_day_of_week'].astype('uint8')
df['dropoff_day_of_month'] =   df['dropoff_day_of_month'].astype('uint8')


df['dropoff_month'] =  df['dropoff_month'].astype('uint8')
df['busyness'] =  df['busyness'].astype('uint8')


In [3]:
!pip install nyoka


In [4]:
from nyoka import skl_to_pmml

In [5]:
import numpy as nump
from sklearn.ensemble import RandomForestClassifier



training_size =  int(0.8 * len(df))

training_df =   df.iloc[:training_size]

testing_df =  df.iloc[training_size:]







X_training = training_df.drop('busyness',  axis=1)
y_training = training_df['busyness']

X_testing = testing_df.drop('busyness',  axis=1)
y_testing = testing_df['busyness']








model = RandomForestClassifier(n_estimators=1, random_state=40)

model.fit(X_training, y_training)




y_predict = model.predict(X_testing)


# make metrics
accuracy =  metrics.accuracy_score(y_testing, y_predict)

f1_score =    metrics.f1_score(y_testing, y_predict, average='macro')
recall =   metrics.recall_score(y_testing, y_predict, average='macro')
precision =   metrics.precision_score(y_testing, y_predict, average='macro')



# print the metrics
print("accuracy:",  accuracy)
print("f1:",   f1_score)
print("recall:",   recall)
print("pecision:",   precision)




accuracy: 0.7555529513575885
f1: 0.7541171350829725
recall: 0.7544918840557091
pecision: 0.7561608186631122


In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


s =  StandardScaler()

pipe = Pipeline(steps=[('scaler', s),
                       ('model', model)])


pipe.fit(X_training, y_training)

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 RandomForestClassifier(n_estimators=1, random_state=40))])

In [7]:
path_pmml =  "/content/drive/MyDrive/model6.91.pmml"

skl_to_pmml(pipeline=pipe,  col_names=X_training.columns.tolist(),  target_name='busyness',  pmml_f_name=path_pmml)

In [8]:
import pickle



with open('/content/drive/My Drive/t6.91random_forest_model.pkl', 'wb') as file:
    pickle.dump(model, file)






In [9]:
i = model.feature_importances_



how_important_df = panda.DataFrame({
    'Feature': X_training.columns,
    'Importance': i
})


how_important_df.sort_values('Importance', ascending=False, inplace=True)


how_important_df.to_csv('/content/drive/My Drive/t6.91feature_importances.csv',  index=False)
